In [2]:
import os
from pprint import pprint


import pymongo
import bson

MONGODB_URI = "MONGODB URI HERE"


# Connect to your MongoDB cluster:
client = pymongo.MongoClient(MONGODB_URI)

# Get a reference to the "sample_mflix" database:
db = client["swymv3premium-01"]

# Get a reference to the "movies" collection:
ActivityStoreCollection = db["SwymActivityStore"]

In [3]:
pipeline= [
    {
        "$match": {
            "pid": 'Byrxkre2+7KfnS1Uy9OKkJPaimHFBDsOoh6QCHM49Co=',
            "rel":'sa',
            "_at":{"$in":["add-list","add-listitem","copy-list-link","add-list-comment","add-listitem-comment","clone-list","reachout-shopper"] },
        },
    },
    
    {"$project":
         {"activity_date":
              {"$dateToString":{"format": '%Y-%m-%d', "date":{"$toDate":"$cts"}}}, 
          "today_date":{"$dateToString":{"format": '%Y-%m-%d', "date":"$$NOW"}}, 
          "activity_uid":"$uid", 
          "activity_type":"$_at", 
          "merchant_id":"$pid", 
          "_id":0}},
    
    
    
    {
        "$lookup":{
            "from": "SwymShopperSummary",
            "let": { "merchant_id":'$merchant_id', "act_uid":"$activity_uid"},
            "pipeline": [
              { "$match":
                 { "$expr":
                    { "$and":
                      [
                         { "$eq": [ "$pid",  "$$merchant_id" ] },
                         { "$eq": ["$uid", "$$act_uid"]},
                         { "$ne": ["$sauid", None] },
                      ]
                    }
                 }
              },
              { "$project": 
               { "_id":0, "shopper_email":"$em", "shopper_name":{"$concat":["$fname", " ", "$lname"]}, "sa_id":"$sauid"}},
            ],
            "as": "shoppers"
         },
    },
    {"$unwind":"$shoppers"},
    
    {
        "$lookup":{
            "from": "SwymMerchantUserStore",
            "let": { "merchant_id":'$merchant_id', "sa_id":"$shoppers.sa_id"},
            "pipeline": [
              { "$match":
                 { "$expr":
                    { "$and":
                      [
                         { "$eq": [ "$pid",  "$$merchant_id" ] },
                         { "$eq": ["$uid", "$$sa_id"]},
                      ]
                    }
                 }
              },
              { "$project": { "_id":0, "sa_email":"$em", "sa_name":{"$concat":["$fname", " ", "$lname"]}, "store_id":"$stid"}},
            ],
            "as": "sa_details"
         },
    },
    {"$unwind":"$sa_details"},
    {"$addFields":{"store_id":"$sa_details.store_id"}},
    {"$addFields":{"sa_name":"$sa_details.sa_name"}},
    {"$addFields":{"sa_email":"$sa_details.sa_email"}},
    
    
    {"$addFields":{"shopper_name":"$shoppers.shopper_name"}},
    {"$addFields":{"shopper_email":"$shoppers.shopper_email"}},
    
    
    {
        "$group": {
            "_id": {
                "sa":"$sa_email",
                "shopper":"$shopper_email",
                "activity_date":"$activity_date"
            },
            "Store ID":{"$first":"$store_id"},
            "SA Name":{"$first":"$sa_name"},
            "SA Email":{"$first":"$sa_email"},
    		"Shopper Name":{"$first":"$shopper_name"},
    		"Shopper Email":{"$first":"$shopper_email"},
    	    "Activity Date": {"$first": "$activity_date"},
    	    "Reporting Date":{"$first":"$today_date"},
    	    "Days Difference":{"$first":{ "$round": { "$divide": [{ "$subtract": [{"$dateFromString":{"dateString":"$today_date"}}, {"$dateFromString":{"dateString":"$activity_date"}}] }, 86400000] }}},
    		"Activities":{"$push":"$activity_type"},
        },
    },
    
    { "$addFields":{
        "Add List": {
            "$filter": {
                "input": "$Activities",
                "as": "data",
                "cond": { "$or": [ {"$eq": ["$$data", "add-list"]}, {"$eq": ["$$data", "clone-list"]}] }
                }
            }
        }
    },
    
    {"$addFields": {"Add List Count":{"$size":"$Add List"}}},
    
    { "$addFields":{
        "Add Listitem": {
            "$filter": {
                "input": "$Activities",
                "as": "data",
                "cond": { "$or": [ {"$eq": ["$$data", "add-listitem"]}] }
                }
            }
        }
    },
    
    {"$addFields": {"Add Listitem Count":{"$size":"$Add Listitem"}}},
    
    { "$addFields":{
        "Add Comments": {
            "$filter": {
                "input": "$Activities",
                "as": "data",
                "cond": { "$or": [ {"$eq": ["$$data", "add-list-comment"]}, {"$eq": ["$$data", "add-listitem-comment"]}] }
                }
            }
        }
    },
    
    {"$addFields": {"Add Comemnt Count":{"$size":"$Add Comments"}}},
    
    { "$addFields":{
        "Reachout Shopper": {
            "$filter": {
                "input": "$Activities",
                "as": "data",
                "cond": { "$or": [ {"$eq": ["$$data", "copy-list-link"]}, {"$eq": ["$$data", "reachout-shopper"]}] }
                }
            }
        }
    },
    {"$addFields": {"Reachout Shopper Count":{"$size":"$Reachout Shopper"}}},
    
    {"$unset":["sa_details", "shoppers", "_id", "merchant_id", "activity_uid"]},
    
]

In [11]:
activityData = ActivityStoreCollection.aggregate(pipeline)

In [15]:
import pandas as pd
actvityDataDf = pd.DataFrame(list(activityData))

In [16]:
actvityDataDf.head(5)

,Store ID,SA Name,SA Email,Shopper Name,Shopper Email,Activity Date,Reporting Date,Days Difference,Activities,Add List,Add List Count,Add Listitem,Add Listitem Count,Add Comments,Add Comemnt Count,Reachout Shopper,Reachout Shopper Count
0,store:021_oakbrook,Katherine Malley,k.malley@untuckit.com,Daniel Rogers,pdw38@aol.com,2021-11-12,2022-03-06,114.0,"[add-list, add-listitem, add-listitem, reachou...",[add-list],1,"[add-listitem, add-listitem]",2,[],0,[reachout-shopper],1
1,store:052_boca_raton,Sergio Gil,s.gil@untuckit.com,Sam Bell,sammythebell@yahoo.com,2021-11-12,2022-03-06,114.0,"[add-list, add-listitem, reachout-shopper]",[add-list],1,[add-listitem],1,[],0,[reachout-shopper],1
2,store:088_costa_mesa,Carlos Uribe,c.uribe@untuckit.com,Tom Rahe,tomrahe@gmail.com,2021-11-15,2022-03-06,111.0,[add-list],[add-list],1,[],0,[],0,[],0
3,store:028_houston,Dolores Rodriguez,d.rodriguez@untuckit.com,Michael Stone,mikestoneatty@hotmail.com,2021-10-26,2022-03-06,131.0,"[add-list, add-listitem]",[add-list],1,[add-listitem],1,[],0,[],0
4,store:084_milwaukee,Kelsey Wrensch,k.wrensch@untuckit.com,Matt Gray,mjg@ymh.com,2021-11-20,2022-03-06,106.0,"[add-list, add-listitem, reachout-shopper]",[add-list],1,[add-listitem],1,[],0,[reachout-shopper],1


In [19]:
actvityDataDf.groupby(["Shopper Name"])["Add Listitem Count"].sum()

Shopper Name
                   0
ANTONIO MANZO     15
Aaron Scofield    15
Abdel Hussein      1
Abel Estopin       1
                  ..
ryan pociask       0
scott Mahan        2
scott humbard      0
sherri navarro     0
todd jacobs        3
Name: Add Listitem Count, Length: 609, dtype: int64

In [22]:
actvityDataDf.groupby(["Shopper Name"])["Add Listitem Count"].sum().to_csv('test.csv')